# Topic Modeling and Latent Dirichlet Allocation (LDA) in Python

Description:
This algorthm categories over a million newspaper headlines using LDA

###### link: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [2]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

print(len(documents))
print(documents.shape)
print(data[:4])
print(documents[:5])



21295
(21295, 2)
   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import *
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zachwolpe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zachwolpe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    # I used PorterStemmer() method as stemmer would through an error

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [18]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']

 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [19]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [20]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0 
for k,v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [54]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Gensim doc2bow

In [28]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(77, 1), (116, 1), (495, 1), (4452, 1)]

In [29]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print('word {} (\"{}\") appears {} times'.format(bow_doc_4310[i][0],
         dictionary[bow_doc_4310[i][0]],
         bow_doc_4310[i][1]))

word 77 ("bushfir") appears 1 times
word 116 ("help") appears 1 times
word 495 ("rain") appears 1 times
word 4452 ("dampen") appears 1 times


## TF-IDF

In [32]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.5633762170099382), (1, 0.40292701761748095), (2, 0.4797511976087543), (3, 0.5386054631865693)]


## Running LDA using Bag of Words

In [33]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [36]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {} \n'.format(idx, topic))

Topic: 0 
Words: 0.011*"plan" + 0.010*"chief" + 0.009*"iraq" + 0.008*"govt" + 0.008*"health" + 0.007*"secur" + 0.007*"call" + 0.006*"test" + 0.005*"sign" + 0.005*"push" 

Topic: 1 
Words: 0.014*"polic" + 0.014*"plan" + 0.013*"govt" + 0.009*"baghdad" + 0.007*"warn" + 0.007*"support" + 0.007*"station" + 0.006*"aceh" + 0.006*"green" + 0.006*"shark" 

Topic: 2 
Words: 0.017*"kill" + 0.014*"council" + 0.009*"group" + 0.008*"consid" + 0.007*"clear" + 0.007*"reject" + 0.007*"govt" + 0.006*"race" + 0.006*"law" + 0.006*"plan" 

Topic: 3 
Words: 0.031*"polic" + 0.015*"investig" + 0.014*"charg" + 0.013*"arrest" + 0.012*"lead" + 0.009*"tour" + 0.008*"boost" + 0.007*"fatal" + 0.007*"accid" + 0.006*"fund" 

Topic: 4 
Words: 0.014*"polic" + 0.014*"crash" + 0.013*"iraq" + 0.011*"bomb" + 0.010*"miss" + 0.010*"say" + 0.009*"iraqi" + 0.008*"govt" + 0.007*"search" + 0.007*"woe" 

Topic: 5 
Words: 0.013*"water" + 0.011*"seek" + 0.009*"charg" + 0.008*"test" + 0.008*"death" + 0.007*"face" + 0.006*"hold" + 0.

## Running LDA using TF-IDF

In [37]:
lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \n Word: {} \n'.format(idx, topic))

Topic: 0 
 Word: 0.008*"crash" + 0.008*"plan" + 0.007*"iraqi" + 0.006*"fund" + 0.006*"feder" + 0.006*"report" + 0.005*"urg" + 0.005*"offer" + 0.005*"govt" + 0.005*"forc" 

Topic: 1 
 Word: 0.011*"court" + 0.009*"water" + 0.008*"troop" + 0.007*"kill" + 0.006*"injur" + 0.005*"raid" + 0.005*"claim" + 0.005*"face" + 0.004*"australian" + 0.004*"talk" 

Topic: 2 
 Word: 0.011*"kill" + 0.010*"face" + 0.010*"govt" + 0.009*"bomb" + 0.006*"court" + 0.006*"bali" + 0.006*"toll" + 0.006*"death" + 0.006*"iraq" + 0.006*"rise" 

Topic: 3 
 Word: 0.009*"charg" + 0.007*"lead" + 0.007*"green" + 0.006*"hop" + 0.006*"govt" + 0.005*"trade" + 0.005*"french" + 0.005*"hous" + 0.005*"say" + 0.005*"australia" 

Topic: 4 
 Word: 0.016*"plan" + 0.010*"protest" + 0.007*"report" + 0.006*"polic" + 0.006*"charg" + 0.006*"urg" + 0.006*"warn" + 0.006*"anti" + 0.006*"world" + 0.005*"miss" 

Topic: 5 
 Word: 0.023*"iraq" + 0.011*"sar" + 0.010*"say" + 0.007*"korea" + 0.006*"plan" + 0.005*"deni" + 0.005*"warn" + 0.005*"engl

## Performance evaluation by classifying sample document using LDA Bag of Words model


In [38]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [39]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4272737503051758	 
Topic: 0.014*"polic" + 0.014*"plan" + 0.013*"govt" + 0.009*"baghdad" + 0.007*"warn" + 0.007*"support" + 0.007*"station" + 0.006*"aceh" + 0.006*"green" + 0.006*"shark"

Score: 0.4126927852630615	 
Topic: 0.014*"polic" + 0.014*"crash" + 0.013*"iraq" + 0.011*"bomb" + 0.010*"miss" + 0.010*"say" + 0.009*"iraqi" + 0.008*"govt" + 0.007*"search" + 0.007*"woe"

Score: 0.02001252956688404	 
Topic: 0.017*"court" + 0.012*"face" + 0.010*"welcom" + 0.010*"work" + 0.010*"murder" + 0.009*"trial" + 0.009*"iraq" + 0.009*"polic" + 0.008*"hear" + 0.007*"probe"

Score: 0.020007606595754623	 
Topic: 0.017*"kill" + 0.014*"council" + 0.009*"group" + 0.008*"consid" + 0.007*"clear" + 0.007*"reject" + 0.007*"govt" + 0.006*"race" + 0.006*"law" + 0.006*"plan"

Score: 0.02000485174357891	 
Topic: 0.016*"fund" + 0.016*"concern" + 0.009*"drought" + 0.007*"health" + 0.007*"secur" + 0.007*"road" + 0.007*"air" + 0.006*"expect" + 0.006*"nation" + 0.006*"round"

Score: 0.02000286988914013	 
To

## Performance evaluation by classifying sample document using LDA TF-IDF model.

In [40]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199392557144165	 
Topic: 0.023*"iraq" + 0.011*"sar" + 0.010*"say" + 0.007*"korea" + 0.006*"plan" + 0.005*"deni" + 0.005*"warn" + 0.005*"england" + 0.005*"road" + 0.005*"spark"

Score: 0.020011579617857933	 
Topic: 0.008*"crash" + 0.008*"plan" + 0.007*"iraqi" + 0.006*"fund" + 0.006*"feder" + 0.006*"report" + 0.005*"urg" + 0.005*"offer" + 0.005*"govt" + 0.005*"forc"

Score: 0.0200111735612154	 
Topic: 0.011*"kill" + 0.010*"face" + 0.010*"govt" + 0.009*"bomb" + 0.006*"court" + 0.006*"bali" + 0.006*"toll" + 0.006*"death" + 0.006*"iraq" + 0.006*"rise"

Score: 0.02000897005200386	 
Topic: 0.017*"council" + 0.016*"polic" + 0.008*"death" + 0.007*"urg" + 0.006*"govt" + 0.006*"boost" + 0.005*"murder" + 0.005*"probe" + 0.005*"continu" + 0.004*"claim"

Score: 0.020007913932204247	 
Topic: 0.016*"plan" + 0.010*"protest" + 0.007*"report" + 0.006*"polic" + 0.006*"charg" + 0.006*"urg" + 0.006*"warn" + 0.006*"anti" + 0.006*"world" + 0.005*"miss"

Score: 0.02000642567873001	 
Topic: 0.009*"ch

## Testing model on unseen document

In [41]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3983001410961151	 Topic: 0.017*"court" + 0.012*"face" + 0.010*"welcom" + 0.010*"work" + 0.010*"murder"
Score: 0.26674753427505493	 Topic: 0.014*"polic" + 0.014*"plan" + 0.013*"govt" + 0.009*"baghdad" + 0.007*"warn"
Score: 0.19493356347084045	 Topic: 0.014*"polic" + 0.014*"crash" + 0.013*"iraq" + 0.011*"bomb" + 0.010*"miss"
Score: 0.020007330924272537	 Topic: 0.013*"water" + 0.011*"seek" + 0.009*"charg" + 0.008*"test" + 0.008*"death"
Score: 0.020005548372864723	 Topic: 0.011*"plan" + 0.010*"chief" + 0.009*"iraq" + 0.008*"govt" + 0.008*"health"
Score: 0.020003486424684525	 Topic: 0.016*"fund" + 0.016*"concern" + 0.009*"drought" + 0.007*"health" + 0.007*"secur"
Score: 0.020000910386443138	 Topic: 0.018*"budget" + 0.014*"govt" + 0.010*"claim" + 0.009*"plan" + 0.008*"final"
Score: 0.020000571385025978	 Topic: 0.017*"kill" + 0.014*"council" + 0.009*"group" + 0.008*"consid" + 0.007*"clear"
Score: 0.020000552758574486	 Topic: 0.010*"rise" + 0.010*"claim" + 0.009*"aussi" + 0.006*"anzac